In [36]:
move_list = ['L', 'R', 'R']
move_encode_dict = {'L': 0, 'R': 1}
move_list_encoded = [move_encode_dict[move] for move in move_list]
move_list_encoded_reversed = list(reversed(move_list_encoded))
sum(v * 2**k for k, v in enumerate(move_list_encoded_reversed)) + 1

4

In [84]:
import functools as ft

@ft.lru_cache
def fusc(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        if n % 2 == 0:
            return fusc(n // 2)
        else:
            return fusc((n + 1) // 2) + fusc((n - 1) // 2)
        
print(fusc(19991 ** 7))

print(fusc.cache_info())

125430397184702359
CacheInfo(hits=100, misses=200, maxsize=128, currsize=128)


In [91]:
import itertools as it
import functools as ft
import operator

class CalkinWilf:
    ROOT_LEVEL = 0
    ROOT_LEVEL_INDEX = 0
    ROOT_MOVE_LIST = []
    ROOT_FRACTION_TUPLE = (1, 1)

    def __init__(self, level=None, level_index=None, move_list=None, fraction_tuple=None):
        if level is None:
            self.level = type(self).ROOT_LEVEL
            self.level_index = type(self).ROOT_LEVEL_INDEX
            self.bfs_index = 2 ** type(self).ROOT_LEVEL + type(self).ROOT_LEVEL_INDEX
            self.move_list = type(self).ROOT_MOVE_LIST
            self.fraction_tuple = type(self).ROOT_FRACTION_TUPLE
        else:
            self.level = level
            self.level_index = level_index
            self.bfs_index = 2 ** self.level + self.level_index
            self.move_list = move_list
            self.fraction_tuple = fraction_tuple
    
    def L(self):
        new_level = self.level + 1
        new_level_index = 2 * self.level_index
        new_move_list = self.move_list + ["L"]
        a, b = self.fraction_tuple
        new_fraction_tuple = (a, a + b)
        return type(self)(new_level, new_level_index, new_move_list, new_fraction_tuple)
    
    def R(self):
        new_level = self.level + 1
        new_level_index = 2 * self.level_index + 1
        new_move_list = self.move_list + ["R"]
        a, b = self.fraction_tuple
        new_fraction_tuple = (a + b, b)
        return type(self)(new_level, new_level_index, new_move_list, new_fraction_tuple)
    
    def P(self):
        if self.level == type(self).ROOT_LEVEL:
            return None
        else:
            new_level = self.level - 1
            new_level_index = self.level_index // 2
            move_list = self.move_list
            new_move_list = move_list[0:-1]
            a, b = self.fraction_tuple
            if a < b:
                new_fraction_tuple = (a, b - a)
            else:
                new_fraction_tuple = (a - b, b)
            return type(self)(new_level, new_level_index, new_move_list, new_fraction_tuple)
    
    def run_list(self):
        return [(k, len(list(g))) for k, g in it.groupby(self.move_list)]
    
    def __repr__(self):
        return f"{type(self).__name__}({self.level}, {self.level_index}, {self.move_list}, {self.fraction_tuple})"
    
    def __eq__(self, other):
        return self.level == other.level and self.level_index == other.level_index
    
    @staticmethod
    def find_level_index(move_list: list[str]):
        move_encode_dict = {'L': 0, 'R': 1}
        move_list_encoded = [move_encode_dict[move] for move in move_list]
        move_list_encoded_reversed = list(reversed(move_list_encoded))
        level_index = sum(v * 2**k for k, v in enumerate(move_list_encoded_reversed)) + 1
        return level_index
    
    @staticmethod
    def find_cf(n: int):
        bits = '1' + f"{n:b}"
        runs = [len(list(g)) for _, g in it.groupby(reversed(bits))]
        if n & 1 == 0:          # even ⇒ initial 1‑run has length 0
            runs.insert(0, 0)
        if len(runs) % 2 and len(runs) > 1:   # drop spurious 0‑run
            runs.pop(-2)
        return runs

    @staticmethod
    def find_convergents(a: list[int]) -> list[tuple[int, int]]:
        N = len(a)
        if N == 0:
            return []
        p_prev2, p_prev1 = 0, 1
        q_prev2, q_prev1 = 1, 0
        convergent_list = []
        for k in range(N):
            p_k = a[k] * p_prev1 + p_prev2
            q_k = a[k] * q_prev1 + q_prev2
            convergent_list.append((p_k, q_k))
            p_prev2, p_prev1 = p_prev1, p_k
            q_prev2, q_prev1 = q_prev1, q_k        
        return convergent_list

    @classmethod
    @ft.lru_cache
    def fusc(cls, n):
        if n == 0:
            return 0
        elif n == 1:
            return 1
        else:
            if n % 2 == 0:
                return cls.fusc(n // 2)
            else:
                return cls.fusc((n + 1) // 2) + cls.fusc((n - 1) // 2)

    @classmethod
    def find_move_list(cls, fraction_tuple, move_list):
        if fraction_tuple == (1, 1):
            return move_list
        else:
            a, b = fraction_tuple
            if a < b:
                new_move_list = ["L"] + move_list
                new_fraction_tuple = (a, b - a)
            else:
                new_move_list = ["R"] + move_list
                new_fraction_tuple = (a - b, b)
            return cls.find_move_list(new_fraction_tuple, new_move_list)
        
    @classmethod
    def fraction_tuple_to_node(cls, fraction_tuple):
        move_list = cls.find_move_list(fraction_tuple, [])
        level = len(move_list)
        level_index = cls.find_level_index(move_list)
        return cls(level, level_index, move_list, fraction_tuple)


if __name__ == "__main__":
    depth = 10
    cw_tree = []

    N0 = CalkinWilf()
    level_node_list = []
    level_node_list.append(N0)
    cw_tree.append(level_node_list)

    for n in range(depth):
        previous_level_node_list = level_node_list
        level_node_list = []
        for N in previous_level_node_list:
            level_node_list.append(N.L())
            level_node_list.append(N.R())
        cw_tree.append(level_node_list)

    cw_bfs_list = ft.reduce(operator.iadd, cw_tree, [])

    cw_bfs_tuple_list = [(k, N.fraction_tuple, N.bfs_index) for k, N in enumerate(cw_bfs_list, start=1)]

    print(cw_bfs_list[41].bfs_index, cw_bfs_list[41])
    print(CalkinWilf.find_move_list((8, 13), []))
    N = CalkinWilf.fraction_tuple_to_node((8, 13))
    print(N)
    print(N.run_list())
    cf = CalkinWilf.find_cf(42)
    print(cf)
    print(CalkinWilf.find_convergents(cf)[-1])
    print(CalkinWilf.fusc(42))
    print(CalkinWilf.fusc.cache_info())

42 CalkinWilf(5, 10, ['L', 'R', 'L', 'R', 'L'], (8, 13))
['L', 'R', 'L', 'R', 'L']
CalkinWilf(5, 11, ['L', 'R', 'L', 'R', 'L'], (8, 13))
[('L', 1), ('R', 1), ('L', 1), ('R', 1), ('L', 1)]
[0, 1, 1, 1, 1, 2]
(8, 13)
8
CacheInfo(hits=4, misses=9, maxsize=128, currsize=9)
